# Choropleth map

- 서울시 유기 동물 지도 
- 지역별 유기동물 수 Heatmap으로 보여주기
- [서울시 동물등록 현황(자치구)](https://opengov.seoul.go.kr/public/26296659)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import folium
from folium.features import DivIcon
import json

In [ ]:
animal = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/프로젝트/유기동물/서울시_동물등록_현황.xlsx', index_col=1)

# 쉼표 제거 후, 자료형 변환
animal['계'] = animal['계'].str.replace(',','').astype(int)

display(animal)

,연번,계,내장형,외장형,인식표
자치구,,,,,
종로구,1,9735,"4,295","4,075","1,365"
중구,2,7938,"4,108","3,357",473
용산구,3,17594,"9,864","6,182","1,548"
성동구,4,17068,"8,672","6,022","2,374"
광진구,5,20316,"10,333","8,684","1,299"
동대문구,6,19243,"8,984","9,139","1,120"
중랑구,7,23561,"11,300","9,207","3,054"
성북구,8,24405,"11,201","11,075","2,129"
강북구,9,18974,"9,319","6,477","3,178"


- https://github.com/southkorea/seoul-maps

In [ ]:
# 서울시 구 경계 정보를 가진 json 파일 불러오기
geo_path = '/content/drive/MyDrive/Colab Notebooks/프로젝트/유기동물/edge.json'

try:
    geo_data = json.load(open(geo_path, encoding='utf-8'))
except:
    geo_data = json.load(open(geo_path, encoding='utf-8-sig'))

# 지도 만들기
s_map = folium.Map(location=[37.564214,127.001699], tiles='Cartodb Positron', zoom_start=12)

# Choropleth로 단계구분도 표시하기
folium.Choropleth(geo_data=geo_data, # 지도 경계
                  data = animal['계'], # 표시하려는 데이터
                  columns = [animal.index, animal['계']], # 열 지정
                  fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
                  key_on='feature.properties.SIG_KOR_NM',
                  ).add_to(s_map)
s_map

# Choropleth map 위에 텍스트 tag 넣기

- [참고 링크](https://yeomss.tistory.com/281)

In [211]:
# 위도경도 매핑
locs = {
    '강남구': (37.49, 127.06),
    '강동구': (37.54, 127.14),
    '강북구': (37.64, 127.01),
    '강서구': (37.55, 126.82),
    '관악구': (37.46, 126.94),
    '광진구': (37.54, 127.08),
    '구로구': (37.49, 126.85),
    '금천구': (37.464505, 126.889922),
    '노원구': (37.65, 127.07),
    '도봉구': (37.67, 127.03),
    '동대문구': (37.58, 127.05),
    '동작구': (37.49, 126.95),
    '마포구': (37.55, 126.9),
    '서대문구': (37.57, 126.93),
    '서초구': (37.47, 127.03),
    '성동구': (37.558, 127.042),
    '성북구': (37.6, 127.01),
    '송파구': (37.5, 127.11),
    '양천구': (37.522974, 126.832303),
    '영등포구': (37.52, 126.9),
    '용산구': (37.534596, 126.961632),
    '은평구': (37.61, 126.92),
    '종로구': (37.581838, 126.97),
    '중구': (37.5619687, 126.980333),
    '중랑구': (37.59, 127.09),
}

In [212]:
for key, value in locs.items():
    folium.map.Marker(
        # 위경도 위치
        [value[0], value[1]],  

        # DivIcon 을 사용
        # html 태그를 이용해서 text를 올릴 수 있음
        icon=DivIcon(
            # icon px 사이즈
            icon_size=(0, 0),
            # icon 좌 상단 위치 설정
            icon_anchor=(0, 0),

            # html 형식으로 text 추가
            # div 태그 안에 style 형식 추가
            html='<div\
                    style="\
                        font-size: 0.8rem;\
                        color: black;\
                        background-color:rgba(255, 255, 255, 0.2);\
                        width:85px;\
                        text-align:center;\
                        margin:0px;\
                    "><b>'
            + key + ': ' + str(animal.loc[key]['계'])
            + '</b></div>',
        )).add_to(s_map)

s_map